### Load Environments

In [ ]:
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
from utils.post_utils import plotPR
from utils.post_utils import res_remapping
from main import ddetr_main
from engine.da_eval import eval_da_DDETR

### Evaluating Deformable DETR HOI Detector on `100DOH` testset

2 ways to generate detection results respect to different sub-criteria
- Specify `eval_constraint=None` in evluaation, and run the next block `res_remapping` to remap the result to sub-criterion
- Directly specify `eval_constraint` to the desired sub-criterion, feasible criteria: [`None` (all included), `'handstate'`, `'handside'`, `'hand+obj'`]

#### Evaluation

In [ ]:
DATAPATH = "./data/100DOH"
task = 'hand_obj'
IS_TRAIN = False    # set to false if evaluation
IS_DEMO = True
cp = '/home/swdev/contactEst/InteractionDetectorDDETR/ddetr_dv/models/checkpoints/DeformableDETR/deform-300-cp-66600/'
scaling_rate = 2
cls_thres:float=0.1
nms_thres:float=0.5
criteria = ['handstate', 'handside', 'hand+obj', None]
eval_constraint = criteria[0]   # Feasible constraints: [None (all included), 'handstate', 'handside', 'hand+obj']

ddetr_main(cp,
            dataset_path=DATAPATH,
            scaling_rate=scaling_rate,
            is_train=IS_TRAIN,
            is_demo=IS_DEMO,
            task=task,
            cls_thres=cls_thres,
            nms_thres=nms_thres,
            ignore_mismatched_sizes=False,
            eval_constraint=eval_constraint)

#### Result remapping
1. Specify the path to detection result
2. Specify the output path
3. Specify the criterion

In [ ]:
res_path = 'results/ddetrhod20/res.json'
output_path = 'results/ddetrhod20'
criterion = criteria[3]

res_remapping(res_path, output_path, criterion)

### Evaluation Result Plotting

In [ ]:
criterion = criteria[2]

gtpath = f"data/json_format/coco_gt/gt_{criterion if criterion else 'all'}.json"
dtpath = f"results/ddetrhod20/res_{criterion if criterion else 'all'}.json"

mdl = 'Deformable DETR 20 Queries'

cocoGt = COCO(gtpath)
cocoDt = cocoGt.loadRes(dtpath)
coco_eval = COCOeval(cocoGt, cocoDt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

plotPR(coco_eval, criterion, mdl)

### Domain Adaptability Evaluation

Load Deformable DETR model

In [ ]:
from engine.load_model import load_model

task = 'hand_obj'
num_cls = 4
scaling_rate = 2
target_size = [1333 // scaling_rate, 800 // scaling_rate]

mdl = 'ddetr'
cp = 'models/checkpoints/DeformableDETR/deform-20-cp-66600'
nr_queries = 300 if cp.split('/')[-2].endswith('RichQueries') else 20

model, cp, cfg = load_model(mdl,
                            cp = cp,
                            task = task,
                            nr_queries = nr_queries, 
                            num_cls = num_cls, 
                            is_train = False)
model = model.cuda().eval()

Evaluating Domain Adaptability of Deformable DETR on the custom dataset and demonstrating the results including plots

In [ ]:
criterion = 'handstate'  # 'hand+obj', 'handstate'
eval_da_DDETR(model, criterion)

### `Faster RCNN` Domain Adaptability Evaluation

In [ ]:
criteria = ['handstate', 'handside', 'hand+obj', None]
criterion = criteria[0]

gtpath = f"/home/swdev/contactEst/InteractionDetectorDDETR/eval/data/anns/gt_{criterion if criterion else 'all'}.json"
dtpath = f"/home/swdev/contactEst/InteractionDetectorDDETR/results/da_res_fr/res_{criterion if criterion else 'all'}.json"

mdl = 'Faster R-CNN + ResNet101'

cocoGt = COCO(gtpath)
cocoDt = cocoGt.loadRes(dtpath)
coco_eval = COCOeval(cocoGt, cocoDt, iouType='bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

plotPR(coco_eval, criterion, mdl)